<a href="https://colab.research.google.com/github/shallex/QA-ruGPT3Small/blob/main/GPT3_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune ruGPT3Small on QA Mail.ru

## Install enviroment

In [ ]:
!pip3 install folium==0.2.1

     |████████████████████████████████| 71kB 11.0MB/s 
  Created wheel for folium: filename=folium-0.2.1-cp37-none-any.whl size=79810 sha256=ba0cc6d9a3ff115e59dd6cd2132559b104519cab70e798e239dd8b8c85862b23
  Stored in directory: /root/.cache/pip/wheels/b8/09/f0/52d2ef419c2aaf4fb149f92a33e0008bdce7ae816f0dd8f0c5
Successfully built folium
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [ ]:
!pip3 install urllib3==1.25.4 --upgrade

     |████████████████████████████████| 133kB 32.9MB/s 
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip3 install transformers==2.8.0

     |████████████████████████████████| 573kB 30.7MB/s 
     |████████████████████████████████| 901kB 50.5MB/s 
     |████████████████████████████████| 133kB 54.7MB/s 
     |████████████████████████████████| 5.6MB 50.9MB/s 
     |████████████████████████████████| 1.2MB 26.0MB/s 
     |████████████████████████████████| 7.6MB 34.5MB/s 
     |████████████████████████████████| 81kB 12.2MB/s 


In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py

--2021-06-07 18:28:21--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34150 (33K) [text/plain]
Saving to: ‘pretrain_transformers.py’

pretrain_transforme 100%[===================>]  33.35K  --.-KB/s    in 0s      

2021-06-07 18:28:21 (116 MB/s) - ‘pretrain_transformers.py’ saved [34150/34150]



In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

--2021-06-07 18:28:21--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10474 (10K) [text/plain]
Saving to: ‘generate_transformers.py’

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      

2021-06-07 18:28:21 (112 MB/s) - ‘generate_transformers.py’ saved [10474/10474]



In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [ ]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 8048, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 8048 (delta 65), reused 97 (delta 44), pack-reused 7913
Receiving objects: 100% (8048/8048), 14.11 MiB | 28.78 MiB/s, done.
Resolving deltas: 100% (5464/5464), done.
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-ullo0jg0
Created temporary directory: /tmp/pip-req-tracker-8yr9ayjp
Created requirements tracker '/tmp/pip-req-tracker-8yr9ayjp'
Created temporary directory: /tmp/pip-install-jkaym86a
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-h75k6lam
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-8yr9ayjp'
    Running setup.py (

## Add data to colab


In [ ]:
data_path = "/content/MAIL_TEXT.txt"
!ls "$data_path"

/content/MAIL_TEXT.txt


## Prepare data

In [ ]:
with open(data_path, "r", encoding = "ISO-8859-1") as file:
    text = file.read()

In [ ]:
topics = []
all_QA = []
for line in text.split("\n"):
    if "Q: " in line and "A: " in line:
        QA_text = line.split("A: ")
        if len(QA_text) == 2:
            qstn = QA_text[0].replace("Q: ", " ").strip()
            ans = QA_text[1].strip()
            QA_text = f"A: {ans}"
            QA_res = f"<s>Q: {qstn}\n{QA_text}</s>"
            all_QA.append(QA_res)
            topics.append(qstn)

In [ ]:
import numpy as np
import random

In [ ]:
random.seed(1234)
np.random.seed(1234)

In [ ]:
len(all_QA)

50497

In [ ]:
valid_size = 50000
print(valid_size)

50000


In [ ]:
unique_topics = list(set(topics))

In [ ]:
valid_topics = []

In [ ]:
idxs = set()
for _ in range(valid_size):
    idx = np.random.randint(0, len(unique_topics))
    idxs.add(idx)
    valid_topics.append(unique_topics[idx])

In [ ]:
import nltk


train = []
valid = []
c = 0
for idx, qa in enumerate(all_QA):
    is_train = True
    if idx in idxs:
        is_train = False
    if is_train:
        train.append(qa)
    else:
        valid.append(qa)
    c+=1
    if c%10000==0:
      print(c)

10000
20000
30000
40000
50000


In [ ]:
len(train), len(valid)

(18711, 31786)

In [ ]:
with open("train.txt", "w") as file:
    file.write("\n".join(train))

In [ ]:
with open("valid.txt", "w") as file:
    file.write("\n".join(valid))

## Run finetuning


In [ ]:
!python pretrain_transformers.py \
    --output_dir=essays_model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --fp16 \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

## Check our model

In [ ]:
## Select topic
for idx in range(len(valid)):
  if "образует его общество" in valid[idx]:
    break

In [ ]:
valid[idx]

'<s>Q: Скажите. Вот в наше время, легче найти порядочную девушку, или для секса, на один раз?\nA: Порядочных совсем почти не осталось, поэтому, начиная поиски, надо быть готовым к тому, что они могут затянуться. По поводу другой группы девушек полностью согласен с остальными :)</s>'

In [ ]:
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=essays_model \
    --k=5 \
    --p=0.95 \
    --length=100

2021-01-28 09:47:04.569563: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
01/28/2021 09:47:06 - INFO - transformers.tokenization_utils -   Model name 'essays_model' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming 'essays_model' is a path, a model identifier, or url to a directory containing tokenizer files.
01/28/2021 09:47:06 - INFO - transformers.tokenization_utils -   Didn't find file essays_model/added_tokens.json. We won't load it.
01/28/2021 09:47:06 - INFO - transformers.tokenization_utils -   loading file essays_model/vocab.json
01/28/2021 09:47:06 - INFO - transformers.tokenization_utils -   loading file essays_model/merges.txt
01/28/2021 09:47:06 - INFO - transformers.tokenization_utils -   loading file None
01/28/2021 09:47:06 - INFO - transformers.tokenization_utils -   loading file essays_model/special_tokens_map.json
01/28/2021 09:47:06 - INFO - tra